# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
city_weather_csv = "../output_data/city_weather.csv"
city_weather_df = pd.read_csv(city_weather_csv)
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Edmundston,47.3737,-68.3251,51.13,83,56,8.66,CA,1631358598
1,Ramon Magsaysay,8.0019,123.5094,75.15,92,92,3.13,PH,1631358618
2,Az Zabadānī,33.7249,36.1005,81.21,32,0,10.36,SY,1631358619
3,Fortuna,40.5982,-124.1573,52.72,75,75,0.00,US,1631358620
4,Saint-Alban,43.7000,1.4167,77.09,64,20,3.44,FR,1631358620
...,...,...,...,...,...,...,...,...,...
592,Wildeshausen,52.9000,8.4333,68.11,88,56,13.38,DE,1631359066
593,Estaires,50.6457,2.7278,69.89,78,100,13.42,FR,1631359067
594,Saguday,16.5428,121.5638,75.18,96,100,2.84,PH,1631359067
595,Gulu,2.7746,32.2990,79.50,60,38,2.19,UG,1631359068


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

 # Store latitude and longitude in locations, store humidity data in humidity
locations = city_weather_df[["Lat", "Lng"]]
humidity = city_weather_df["Humidity"]

In [53]:
 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
narrow_cities = city_weather_df.loc[city_weather_df["Max Temp"]>=70]
narrow_cities = narrow_cities.loc[narrow_cities["Max Temp"]<80]
narrow_cities = narrow_cities.loc[narrow_cities["Wind Speed"]<=10]
narrow_cities = narrow_cities.loc[narrow_cities["Cloudiness"]==0]
narrow_cities.dropna(inplace=False)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,Ayvacık,39.6011,26.4047,77.52,30,0,6.73,TR,1631358623
13,Piastów,52.1843,20.8395,77.05,47,0,9.22,PL,1631358627
42,Topola,44.2542,20.6825,79.88,17,0,6.98,RS,1631358649
121,Lagodekhi,41.8268,46.2767,79.75,46,0,5.30,GE,1631358708
126,Leavenworth,39.2333,-95.0336,74.25,80,0,1.99,US,1631358712
137,Escoublac,47.2947,-2.3596,70.97,61,0,3.44,FR,1631358721
213,Uiwang,37.3449,126.9690,76.32,75,0,3.27,KR,1631358777
249,Buchrain,47.0860,8.3462,77.00,65,0,0.00,CH,1631358805
250,Herbignac,47.4482,-2.3181,71.19,60,0,3.44,FR,1631358807
254,Pornichet,47.2668,-2.3379,71.29,61,0,3.44,FR,1631358809


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [79]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": narrow_cities[["Lat", "Lng"]]
    "rankby": "distance",
    "type": "hotel",
    "key": g_key
}
response = requests.get(base_url, params=params).json()



SyntaxError: invalid syntax (<ipython-input-79-e94ce05cfa2e>, line 4)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
